In [2]:

!pip install beautifulsoup4
!pip install requests
!pip install spacy
!pip install trafilatura


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for urllib3<3,>=1.26 from https://files.pythonhosted.org/packages/37/dc/399e63f5d1d96bb643404ee830657f4dfcf8503f5ba8fa3c6d465d0c57fe/urllib3-2.0.5-py3-none-any.whl.metadata
   ---------------------------------------- 123.8/123.8 kB ? eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 2.0.5 which is incompatible.


In [4]:
from bs4 import BeautifulSoup
import json
import os
import numpy as np
import requests
import re
from requests.models import MissingSchema
import spacy
import trafilatura


In [5]:
urls = ['https://website.understandingdata.com/',
        'https://sempioneer.com/', ]
data = {}

for url in urls:
    # 1. Obtain the response:
    resp = requests.get(url)

    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url] = resp.text


In [10]:
def beautifulsoup_extract_text_fallback(response_content):
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style', ]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def extract_text_from_single_web_page(url):
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text'], json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return resp.content.decode('utf-8'), beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan


single_url = 'http://www.fsb.ru/'
page, text = extract_text_from_single_web_page(url=single_url)
#print(page)
all_relative = re.findall('href=(\'|")(.*?)(\'|")', page)
for aa in all_relative:
    if 'http' in aa[1]:
        continue
    print(f'rlt: {aa[1]}')

all_abs = re.findall('href=(\'|")(http.*?)(\'|")', page)
for aa in all_relative:
    if not 'http' in aa[1]:
        continue
    print(f'abs: {aa[1]}')


rlt: fsb/rss.htm%21contenttype%3Dxml.html
rlt: styles/fsb2019/bootstrap.min.css
rlt: styles/fsb2019/all.css
rlt: styles/fsb/ie_fix.css
rlt: fsb.htm
rlt: fsb/leadership.htm
rlt: fsb/structure.htm
rlt: fsb/supplement.htm
rlt: fsb/press.htm
rlt: fsb/gosuslugi.htm
rlt: fsb/corruption.htm
rlt: javascript:
rlt: javascript:
rlt: javascript:
rlt: fsb/webreception.htm
rlt: fsb/press/message/single.htm%21id%3D10439822%40fsbMessage.html
rlt: fsb/press/message/single.htm%21id%3D10439821%40fsbMessage.html
rlt: fsb/press/message/single.htm%21id%3D10439818%40fsbMessage.html
rlt: fsb/press/message/single.htm%21id%3D10439815%40fsbMessage.html
rlt: fsb/press/message.htm
rlt: fsb/index/easy.htm
rlt: fsb/supplement.htm
rlt: fsb/supplement/contact.htm
rlt: fsb/supplement/contact_ter_org.htm
rlt: fsb/supplement/contantr/mm.htm
rlt: fsb/supplement/employ.htm
rlt: fsb/supplement/employ/doc1.htm
rlt: fsb/supplement/employ/doc2.htm
rlt: fsb/supplement/employ/doc3.htm
rlt: fsb/supplement/employ/doc4.htm
rlt: fsb

F:\fux-issues-bot\venv\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if sys.path[0] == "":
